<a href="https://colab.research.google.com/github/ikoojos/Algorithm-Debt-Research/blob/master/LR_TFIDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
print(sys.path)


['/content', '/env/python', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.10/dist-packages/IPython/extensions', '/usr/local/lib/python3.10/dist-packages/setuptools/_vendor', '/root/.ipython']


In [3]:
from google.colab import drive
drive.mount('/content/drive')

# Add the shared module folder to Python's path
import sys
sys.path.append('/content/drive/My Drive/AD Final Experiments')

import importlib
import preprocessing
importlib.reload(preprocessing)
from preprocessing import preprocess_data

import splitting
importlib.reload(splitting)
from splitting import split_data

import utils
importlib.reload(utils)
from utils import *

import evaluate_model
importlib.reload(evaluate_model)
from evaluate_model import evaluate_best_model

# Example usage
file_path = '/content/drive/My Drive/AD Identification using SATD/liu_datset_processed.csv'
data = preprocess_data(file_path)
X_train_final, X_val, X_test, y_train_final, y_val, y_test = split_data(data)

print("Data preprocessing and splitting complete!")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Data preprocessing and splitting complete!


In [4]:
from itertools import product
from sklearn.linear_model import LogisticRegression

# Preprocess the data
file_path = '/content/drive/My Drive/AD Identification using SATD/liu_datset_processed.csv'
data = preprocess_data(file_path)  # Use preprocess_data to preprocess the dataset

# Split the data into training, validation, and test sets
X_train_final, X_val, X_test, y_train_final, y_val, y_test = split_data(data)  # Use split_data to split data


In [5]:
from sklearn.metrics import confusion_matrix, classification_report

# Your existing code
param_grid = {
    'C': [0.01, 1, 10],
    'penalty': ['l2'],
    'max_iter': [1, 10, 100, 200]
}

best_score = -1
best_params = None
best_model = None

for C, penalty, max_iter in product(param_grid['C'], param_grid['penalty'], param_grid['max_iter']):
    if penalty == 'elasticnet':
        solver = 'saga'
    else:
        solver = 'lbfgs'

    try:
        pipeline = Pipeline([
            ('TFIDF', TfidfVectorizer()),
            ('scaler', StandardScaler(with_mean=False)),
            ('clf', LogisticRegression(C=C, penalty=penalty, max_iter=max_iter, solver=solver, random_state=42, class_weight='balanced'))
        ])

        pipeline.fit(X_train_final, y_train_final)
        y_val_pred = pipeline.predict(X_val)
        score = accuracy_score(y_val, y_val_pred)

        if score > best_score:
            best_score = score
            best_params = {'C': C, 'penalty': penalty, 'max_iter': max_iter}
            best_model = pipeline

    except Exception as e:
        print(f"Skipping configuration C={C}, penalty={penalty}, max_iter={max_iter} due to error: {e}")

# Call the function
evaluate_best_model(best_model, best_params, best_score, X_test, y_test)

Best parameters found: {'C': 0.01, 'penalty': 'l2', 'max_iter': 200}
Validation set accuracy: 0.8454983922829582

Test Confusion Matrix:
[[  88    2    2   65    1    9    0   33]
 [   2   43    2   26    0    5    2    9]
 [   1    5   59   42    0    3    4   21]
 [  41   14   38 1752    8  119   22  212]
 [   0    0    0    5   12    4    0    2]
 [   6    2    9   92    3  229    5   41]
 [   3    2    2   30    1   12   68   25]
 [  22    4   13  182    5   10   15 4341]]

Test Classification Report:
                        precision    recall  f1-score   support

             ALGORITHM       0.54      0.44      0.48       200
         COMPATIBILITY       0.60      0.48      0.53        89
                DEFECT       0.47      0.44      0.45       135
                DESIGN       0.80      0.79      0.80      2206
         DOCUMENTATION       0.40      0.52      0.45        23
        IMPLEMENTATION       0.59      0.59      0.59       387
                  TEST       0.59      0